In [22]:
group = "gh"
group_id = 15
from_id = 57
to_id = 60
target_directory = f"segments_ns_{group}_{from_id}_{to_id}"
last_db_id = 214531

In [15]:
import os
def getTimeSpan(filename):
    filename = os.path.splitext(os.path.basename(filename))[0]
    try:
        if "_to_" in filename:
            start, end = filename.split("_to_")
            start = start.split("_")[-1]
            end = end.split("_")[0]
            end = float(end)
            start = float(start)
            return (end - start)/1000
        else:
            start, end = filename.split("-")
            start = start.split("_")[-1]
            end = end.split("_")[0]
            end = float(end)
            start = float(start)
            return (end - start)
    except Exception as err:
        print(f"filename is:'{filename}'. Could not parse to get time span.")
        return 0
    
getTimeSpan("STT_AB00094_0057_290646_to_295860.mp3")
getTimeSpan("STT_AB00094_0057_290646_to_295860")

5.214

In [16]:
# import pandas as pd
# # Set Header Names
# all_tasks = "/home/monlamai/all.csv"
# columns = [
#     'id',
#     'group_id',
#     'state',
#     'inference_transcript',
#     'transcript',
#     'reviewed_transcript',
#     'final_transcript',
#     'file_name',
#     'url',
#     'transcriber_id',
#     'transcriber',
#     'reviewer_id',
#     'reviewer',
#     'final_reviewer_id',
#     'final_reviewer',
#     'created_at',
#     'submitted_at',
#     'reviewed_at',
#     'finalised_reviewed_at',
#     'duration',
#     'audio_duration'
#     ]

# df = pd.read_csv(all_tasks, header=None, names=columns)
# exist_files = set(df.file_name)

In [17]:
import os
rows = []
from transformers import pipeline
generator = pipeline(model="openpecha/wav2vec2_run8")

for root, _, files in os.walk(target_directory):
        for file in files:
            # if file in exist_files:
            #     continue
            source_file_path = os.path.join(root, file)
            opt = generator(f"/media/monlamai/SSD/GitHub/split/STT_NS/{target_directory}/{file}")
            inf = opt['text']
            print(inf)
            rows.append([file, f"https://d38pmlk0v88drf.cloudfront.net/stt_pecha_tools/{file}", inf , getTimeSpan(file)])

སོ་སོ་ཚང་ྱོ་མ་མེད་་རེད་ེ།རེད་རེད།
ད་ལྟ་ཁྱེད་རང་ལ་ད་སློབ་ཚན་མང་དྲུག་ཁྱུར་ཡོད་རེད་། ཨ། དེའི་ནང་ནས་ཡང་
སྤྱིར་བཏང་ཡིན་ན་མང་ཆེ་ས་དུ་གཉིས་འདྲ་བོ་རེད་འདུག་ཨ། ཨུ། ད་ཁ་ཤས་ཡིན་ན་
དུན་ད་ད་གདུང་ཙར་ཡ་དེ་སྙིང་རྗེ་དེ་ག་རང་གྲོངས་རེད།
བརྒྱ་ཆ་མངག་ཡོད་པ་ཡིན་ན་ད་ཁོ་ཚོགི་རྩིག་ྒྱྱབརཀྱ་ནས་ཡོདབད་མཐའ་མ་དེ་ལ་ཁོ་ཚོ་གི་ད་ག་རེ་ལབ་དགོས་རེད་ལན་ཞིག་སླེབས་དགོས་རེད་།།
དམུལ་མཐུ་ཤོས་དེ་ག་རེ་རེད་ཤག་ལབ་ན་སིད་ཡེ་རེད་ཤག ཨ་ལེ། རའི་ཀོརི་ཀོ་ན་ལེབས་འདུག་ཡོད་བ།དེ་དེ་ཧང་སང་
སྤྱི་ཚོགས་ག་འདྲའི་ཡིན་ན་རེད།ད་མིང་མང་བ་དེ་ག་རེ་བྱེད་ཀི་འདུག
ད་མ་ཐུག་ནས་འདུག་ཅེ་རེད་ལཡིན་ཡ་ག་ོ་གཅིག་བྱས་ཅི་ཡོད་། དེ་སོ་དྲ་་འཛིན་རྐྱང་བར་ོ་ཡོདཨེ་འདྲའི་འདྲ་བོ་ཐང་མྱོང་ཡོདཡིན་ནས་ང་རང་ཐ་སྙིད་ད་མེད་ཡོད་བ།
ལོ་ག་སྟོན་དགོས་རེད། ཨེ། ཨེདུས་ཚོད་དེ་དུས་
འཆང་བ་ཡིན་ན་ཁྱེད་རང་ལ་ཡར་ལ་རག་ཡ་དང་།ཨོ་དེ་རེ་འདུགཨེ་ནས་མ་འོངས་པ་སྡུག་བསྔལ་དེ་ཧ་པོ་
ཧ་ལོ་ལེབ་ན་ག་རེ་ལབ་རེད་ད། རྒྱུ་རིང་པོ་ཅིག་ལ་གཅིག་རང་ཟེར་ཡ་ད་དེ་ལྷོད་ལྷོད་ཅིག་བཏུང་བཅུག་ད།
ཨེ་ནས་དབྱིན་ཇི་ལེན་པ་ཡིན་ན་སྤྱིར་ད་དབྱིན་ཇིའི་འདུག་ཡང་རྒྱ་གར་ཀ་ཅི་ལོརྒྱུས་
དེང་སང་ཡིན་ནས་ད་ར་ནས་དེ་ཡོད་ན་དངོས་གནས་བྱད་ན་
ད་དེ་ཚོ་ཆགས་འདུག་བསམ་གི་འདུག། ཨོ

In [18]:
rows[0]

['STT_NS0057_0108_1058426_to_1060570.wav',
 'https://d38pmlk0v88drf.cloudfront.net/stt_pecha_tools/STT_NS0057_0108_1058426_to_1060570.wav',
 'སོ་སོ་ཚང་ྱོ་མ་མེད་་རེད་ེ།རེད་རེད།',
 2.144]

In [19]:
import pandas as pd
df = pd.DataFrame(rows, columns =['file_name', 'url', 'inference_transcript', 'audio_duration'])

In [20]:
df[['inference_transcript','url']].iloc[0:10].to_dict()

{'inference_transcript': {0: 'སོ་སོ་ཚང་ྱོ་མ་མེད་་རེད་ེ།རེད་རེད།',
  1: 'ད་ལྟ་ཁྱེད་རང་ལ་ད་སློབ་ཚན་མང་དྲུག་ཁྱུར་ཡོད་རེད་། ཨ། དེའི་ནང་ནས་ཡང་',
  2: 'སྤྱིར་བཏང་ཡིན་ན་མང་ཆེ་ས་དུ་གཉིས་འདྲ་བོ་རེད་འདུག་ཨ། ཨུ། ད་ཁ་ཤས་ཡིན་ན་',
  3: 'དུན་ད་ད་གདུང་ཙར་ཡ་དེ་སྙིང་རྗེ་དེ་ག་རང་གྲོངས་རེད།',
  4: 'བརྒྱ་ཆ་མངག་ཡོད་པ་ཡིན་ན་ད་ཁོ་ཚོགི་རྩིག་ྒྱྱབརཀྱ་ནས་ཡོདབད་མཐའ་མ་དེ་ལ་ཁོ་ཚོ་གི་ད་ག་རེ་ལབ་དགོས་རེད་ལན་ཞིག་སླེབས་དགོས་རེད་།།',
  5: 'དམུལ་མཐུ་ཤོས་དེ་ག་རེ་རེད་ཤག་ལབ་ན་སིད་ཡེ་རེད་ཤག ཨ་ལེ། རའི་ཀོརི་ཀོ་ན་ལེབས་འདུག་ཡོད་བ།དེ་དེ་ཧང་སང་',
  6: 'སྤྱི་ཚོགས་ག་འདྲའི་ཡིན་ན་རེད།ད་མིང་མང་བ་དེ་ག་རེ་བྱེད་ཀི་འདུག',
  7: 'ད་མ་ཐུག་ནས་འདུག་ཅེ་རེད་ལཡིན་ཡ་ག་ོ་གཅིག་བྱས་ཅི་ཡོད་། དེ་སོ་དྲ་་འཛིན་རྐྱང་བར་ོ་ཡོདཨེ་འདྲའི་འདྲ་བོ་ཐང་མྱོང་ཡོདཡིན་ནས་ང་རང་ཐ་སྙིད་ད་མེད་ཡོད་བ།',
  8: 'ལོ་ག་སྟོན་དགོས་རེད། ཨེ། ཨེདུས་ཚོད་དེ་དུས་',
  9: 'འཆང་བ་ཡིན་ན་ཁྱེད་རང་ལ་ཡར་ལ་རག་ཡ་དང་།ཨོ་དེ་རེ་འདུགཨེ་ནས་མ་འོངས་པ་སྡུག་བསྔལ་དེ་ཧ་པོ་'},
 'url': {0: 'https://d38pmlk0v88drf.cloudfront.net/stt_pecha_tools/STT_NS0057_0108_1058426_to_1060570.wav',
  1: 'https://d38pmlk0v88drf.c

In [21]:
df['group_id'] = group_id
df['state'] = 'imported'

In [23]:
df['id'] = df.index + last_db_id + 1

In [14]:
# total = len(df)
# print(total)

# half = int(total/2)
# print(half)

# quarter = int(total/4)
# print(quarter)

388
194
97


In [ ]:
# df[:1000].to_csv(f"segments_p1_ns_{group}_inf.csv", index=False)
# df[1000:2000].to_csv(f"segments_p2_ns_{group}_inf.csv", index=False)
# df[2000:3000].to_csv(f"segments_p3_ns_{group}_inf.csv", index=False)
# df[3000:4000].to_csv(f"segments_p4_ns_{group}_inf.csv", index=False)
# df[4000:5000].to_csv(f"segments_p5_ns_{group}_inf.csv", index=False)
# df[5000:6000].to_csv(f"segments_p6_ns_{group}_inf.csv", index=False)
# df[6000:7000].to_csv(f"segments_p7_ns_{group}_inf.csv", index=False)
# df[7000:8000].to_csv(f"segments_p8_ns_{group}_inf.csv", index=False)
# df[8000:].to_csv(f"segments_p9_ns_{group}_inf.csv", index=False)

# df[:1000].to_csv(f"segments_p1_ns_{group}_inf.csv", index=False)
# df[1000:].to_csv(f"segments_p2_ns_{group}_inf.csv", index=False)

In [ ]:
# batches = total // 1000

# for i in range(batches):
#     df[i*1000:(i+1)*1000].to_csv(f"segments_p{i+1}_ns_{group}_inf.csv", index=False)

# df[batches*1000:].to_csv(f"segments_p{batches+1}_ns_{group}_inf.csv", index=False)

In [24]:
df.head()

,file_name,url,inference_transcript,audio_duration,group_id,state,id
0,STT_NS0057_0108_1058426_to_1060570.wav,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,སོ་སོ་ཚང་ྱོ་མ་མེད་་རེད་ེ།རེད་རེད།,2.144,15,imported,214532
1,STT_NS0058_0071_911783_to_915537.wav,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,ད་ལྟ་ཁྱེད་རང་ལ་ད་སློབ་ཚན་མང་དྲུག་ཁྱུར་ཡོད་རེད་...,3.754,15,imported,214533
2,STT_NS0060_0050_726131_to_728808.wav,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,སྤྱིར་བཏང་ཡིན་ན་མང་ཆེ་ས་དུ་གཉིས་འདྲ་བོ་རེད་འདུ...,2.677,15,imported,214534
3,STT_NS0060_0032_455691_to_458353.wav,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,དུན་ད་ད་གདུང་ཙར་ཡ་དེ་སྙིང་རྗེ་དེ་ག་རང་གྲོངས་རེད།,2.662,15,imported,214535
4,STT_NS0058_0057_692790_to_700674.wav,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,བརྒྱ་ཆ་མངག་ཡོད་པ་ཡིན་ན་ད་ཁོ་ཚོགི་རྩིག་ྒྱྱབརཀྱ་...,7.884,15,imported,214536


In [13]:
df.to_csv(f"segments_ns_{group}.csv", index=False)